# __Part I: Data Retrieval__

## 1 Import Libraries

In [12]:
import os
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from google.cloud import translate
import time

## 2 Define URL for data retrieval

In [ ]:
# Specific URL for comments retreival (Ramathibodi, Siriraj, Chula)
urls = {}
urls['rama'] = "https://www.honestdocs.co/hospitals/ramathibodi-hospital"
urls['siriraj'] = "https://www.honestdocs.co/hospitals/siriraj-hospital"
urls['chula'] = "https://www.honestdocs.co/hospitals/king-chulalongkorn-memorial-hospital"

## 3 Set environment for the Application Default Credentials (ADC)

&emsp;&emsp;&emsp; Prior to use Translation Client Libraries for the Cloud Translation API of Google APIs, you need to register and set the environment for the Application Default Credentials (ADC). [Ref](https://cloud.google.com/translate/docs/quickstart-client-libraries)

In [13]:
# Set up the environment
# ADC_path = r"ADC file path.json"
ADC_path = r"C:\Users\DoZoYo\OneDrive - Mahidol University\My Project-ac0ffafb33bc.json"
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = ADC_path

## 4 Create methods

In [57]:
# 1 Method for commenct retrieval
def retreive_comment(url):
    comment = []
    score = []
    for i in range(1,100):
        
        # Request for the defined URL
        r = requests.get(url, params=dict(query='web scraping',page=i))
        
        # Extracting text from html
        soup = BeautifulSoup(r.text,'html.parser')
        
        # Count total number of comments
        n = len(soup.find_all('div',{'class':'comments__content'}))
        
        # Extracting somment and score for each comments
        for i in range(0,n):
            comment.append(soup.find_all('div',{'class':'comments__content'})[i].get_text().lstrip().strip())
            score.append(soup.find_all('span',{'class':'stars star-rating'})[i].attrs['data-score'])
            i +=1
        i +=1
        if len(soup.find_all('div',{'class':'comments__content'})) <= 0:
            break
    return comment, score


# 2 Method for data translation to english
def translate_eng(comment):
    comment_eng = []
    
    # Use google.cloud to translate Thai to English comment
    translator = translate.Client()
    
    for i in range(0,len(comment)):
        comment_eng.append(translator.translate(comment[i],target_language='en')['translatedText'])
        i +=1
        time.sleep(0.4)
    return comment_eng


# 3 Method for dataframe concatenation
def dataframe_cm(comment, comment_eng, score):
    comment_score = pd.concat([pd.DataFrame(comment, columns=['comment_th']),
                               pd.DataFrame(comment_eng, columns=['comment_eng']),
                               pd.DataFrame(score, columns=['score'], dtype=int)],
                              axis=1)
    corpus = comment_score.drop_duplicates()
    return corpus


# 4 Method for running all above methods then put into csv file
def main_retrieve(url):
    comment, score = retreive_comment(url)
    comment_eng = translate_eng(comment)
    corpus = dataframe_cm(comment, comment_eng, score)
    corpus['comment_th'] = corpus.comment_th.str.replace('\r', '')
    corpus['comment_eng'] = corpus.comment_eng.str.replace('\r', '')
    return corpus

## 4 Data retrieval

In [65]:
# Retrieve comments and translate from Thai to English language

corpus_rama = main_retrieve(urls['rama'])
corpus_siriraj = main_retrieve(urls['siriraj'])
corpus_chula = main_retrieve(urls['chula'])

In [69]:
# Store the corpus to csv file for further analysis

corpus_rama.to_csv(r"C:\Users\DoZoYo\OneDrive - Mahidol University\611 Advanced ML\Sentiment Analysis\data\corpus_rama.csv",
                   encoding='utf-8', sep='|', header=True, index=False)
corpus_siriraj.to_csv(r"C:\Users\DoZoYo\OneDrive - Mahidol University\611 Advanced ML\Sentiment Analysis\data\corpus_siriraj.csv",
                      encoding='utf-8', sep='|', header=True, index=False)
corpus_chula.to_csv(r"C:\Users\DoZoYo\OneDrive - Mahidol University\611 Advanced ML\Sentiment Analysis\data\corpus_chula.csv",
                    encoding='utf-8', sep='|', header=True, index=False)

In [70]:
corpus_rama

,comment_th,comment_eng,score
0,เคยได้ไปลองใช้บริการคลีนิคนอกเวลาของรามาเมื่อต...,Used to try Ramadan part-time clinic earlier t...,5
1,ผมเคยไปทำเลสิกที่นี้ ดีมากครับตอนนี้ผ่ามาจะ2...,"I used to go to LASIK here. Very good. Now, it...",4
2,รักษามาหลายปีแล้วตั้งแต่อายุ14 ตอนนี้จะจบปริญญ...,Maintained for many years since the age of 14 ...,5
3,ที่ตึกพระเทพชั้น 2 สถานที่เครื่องมือดูทันสมัย ...,"At the 2nd floor of Phra Thep building, the pl...",3
4,Its a public hospital so service is bad (nurse...,Its a public hospital so service is bad (nurse...,4
5,เพื่อนของดิฉันเป็นโรคเกี่ยวกับปอด ปอดรั่ว หมอท...,"My friend is a lung disease, a leaky lung, a d...",5
6,วันนั้นไปตรวจเนื่องจากปัญหาการนอนกรน ที่โรงพยา...,That day went to check because of snoring prob...,5
7,แนะนำโรงพยาบาลรามาธิบดีเคยผ่าตัดส่องกล้องช็อคโ...,Introduction to Ramathibodi Hospital Have surg...,5
8,พาย่ามารักษาโรคไต โรงพยาบาลให้การต้อนรับดีมาก ...,Paya to treat kidney disease The hospital is v...,5
9,รักษาเนื้องอกที่มดลูกที่โรงพยาบาลรามาพรีเมียม ...,Treatment of uterine fibroids at Rama Premium ...,5


In [71]:
corpus_siriraj

,comment_th,comment_eng,score
0,เป็นโรงพยาบาลที่ดีมากๆคะเคยรักษาเนื้องอกที่มดล...,It is a very good hospital. I used to treat tu...,5
1,รพ.ศิริราช คุณหมอและพยาบาลน่ารักมากค่ะ ให้คำแน...,"Siriraj Hospital, doctor and nurse are very cu...",5
2,พาคุณย่าไปรักษาโรคมะเร็ง บริการดี รวดเร็วเป็นก...,"Take Grandma to cure cancer, good service, fas...",5
3,พาคุณแม่ไปคลินิกนอกเวลา สูตินรีเวช ขั้นตอนคือ ...,Take Mom to go to the Obstetrics and Gynecolog...,5
4,หน่วยจิตเวชศาสตร์ให้การดูแลผู้ป่วยที่มีปัญหาด้...,The psychiatry unit provides care for patients...,5
5,ให้การรักษาดี รวดเร็ว มีนักศึกษาแพทย์มาร่วมวิน...,"Provide good treatment, fast, have medical stu...",5
6,ที่นี่บริการดีมากเลยครับ คนเยอะแค่ไหนเจ้าหน้าท...,"This service is very good. How many people, th...",5
7,เรารักษาที่นี้ครั้งแรกด้วยอาการซึมเศร้า พยาบาล...,We treat this place for the first time with de...,5
8,บริการเยี่ยมรู้สึกปลอดภัยเมื่อเดินเข้าโรงพยาบา...,Great service. Feel safe when walking into thi...,5
9,เราเกิดที่ รพ นี้ค่ะ สมัยก่อนแม่บอกเราว่าคุณหม...,We were born at this hospital. In the old days...,5


In [72]:
corpus_chula

,comment_th,comment_eng,score
0,บริการค่อยข้างดีค่ะ แต่รอคิวนานนิดหน่อย แต่วิน...,Service is good. But wait for a long time But ...,5
1,ได้มีอาการอาหารเป็นพิษและเข้าทำการรักษาที่โรงพ...,Have food poisoning and take treatment at this...,5
2,มีบริการดีในระดับนึงเลยครับ เรยไปอยู่หลายครั้ง...,There is a good service at a certain level. Ma...,5
3,สำหรับโรงพยาบาลจุฬาลงกรณ์ บริการดีมากค่ะ ฝากท้...,For Chulalongkorn Hospital Very good service L...,5
4,ตรวจดีๆมากๆนะ คุณหมอให้คะแนำนะวินิจฉัย​ดีมากๆ ...,Check it very well The doctor gave me a very g...,5
5,ตรวจดีมากๆค่ะคุณหมอละเอียด พยาบาลก็ดีดูแลคนไข้...,"Very good check Nursing is good, take good car...",4
6,บิรการดีในระดับนึงเลยครับ เรยไปอยู่หลายครั้ง ป...,Good service at a certain level Many times Imp...,4
7,มีโอกาสได้ไปหาหมอที่โรงพยาบาลจุฬาลงกรณ์ค่ะ คุณ...,Had the opportunity to visit a doctor at Chula...,4
8,ตรวจดีมากๆค่ะคุณหมอละเอียด พยาบาลก็ดีดูแลคนไข้...,"Very good check Nursing is good, take good car...",5
9,ไปฝังยาคุมที่โรงพยาบาลจุฬามาค่ะเดินทางสะดวกอยู...,Went to bury the control pills at Chula Hospit...,5
